# Example Code of MANO Hand Pose Estimation by 3D keypoints

### Initialize the MANO layer

- Load MANO shape parameters from file
- Load Target 3D keypoints from file
- Initialize the MANO layer
- Define the `mano_layer_forward` function to predict the MANO 3D joints from input MANO shape and pose parameters
- Define the loss function `MSE` to calculate the difference between the predicted 3D keypoints and the target 3D keypoints
- Initialize the optimizer `Adam` to minimize the loss function
- Run the training loop to optimize the MANO shape and pose parameters
- Visualize the loss curve and the predicted 3D keypoints

### Load MANO shape parameters from file

In [ ]:
import torch
from manopth.manolayer import ManoLayer
from commons import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
data = read_data_from_json(PROJ_ROOT / "data/calibration/mano/subject_7/mano.json")
print(f"Load data from json:\n {data}")

# create a hand pose parameter tensor
shape_params = torch.tensor(data["betas"], dtype=torch.float32).to(device)
print(f"shape_params tensor:\n {shape_params}")

# add batch dimension
shape_params = shape_params.unsqueeze(0)
print(f"shape_params tensor with batch dim:\n {shape_params.shape}")

### Load Target 3D keypoints from file

In [ ]:
kpts_3d = np.load(PROJ_ROOT / "data/recordings/20231022_193630/hand_keypoints_3d.npy")
print(f"3D keypoints:\n {kpts_3d.shape}") # (frames_num, hand_num, joints_num, 3)

# load frame_id=300 keypoints of the left hand (index=1)
target_kpts = kpts_3d[300, 1]
print(f"target_kpts:\n {target_kpts}")

# transform keypoints to tensor
target_kpts = torch.tensor(target_kpts, dtype=torch.float32).to(device)
print(f"target_kpts tensor:\n {target_kpts.shape}")

# add batch dimension
target_kpts = target_kpts.unsqueeze(0)
print(f"target_kpts tensor with batch dim:\n {target_kpts.shape}")

### Initialize the MANO layer

In [ ]:
# Create a MANO layer with default settings
mano_layer = ManoLayer(
    center_idx=0,
    flat_hand_mean=True,
    ncomps=45,
    side="left",
    mano_root=PROJ_ROOT/ "config/mano_models",
    use_pca=True,
    root_rot_mode="axisang",
    joint_rot_mode="axisang",
    robust_rot=True,
)

# Move the MANO layer to the device
mano_layer = mano_layer.to(device)

### Define the `mano_layer_forward` function

In [ ]:
def mano_layer_forward(mano_layer, mano_pose, mano_shape):
    # Forward pass through the MANO layer
    verts, joints = mano_layer(mano_pose[:, :48], mano_shape, mano_pose[:, 48:])
    verts /= 1000.0
    joints /= 1000.0
    return verts, joints

### Initialize the optimizer `Adam`

In [ ]:
# Initialize the MSE loss function
mse_loss = torch.nn.MSELoss(reduction="sum")

# Initialize the MANO pose to optimize by setting the wrist to the target keypoint
optim_pose = torch.zeros((1, 51), dtype=torch.float32, device=device)
optim_pose[:, 48:] = target_kpts[:, 0, :].clone()
print(f"optim_pose initialized:\n {optim_pose}")
# Require gradient for the optimization
optim_pose.requires_grad = True
print(f"optim_pose requires_grad:\n {optim_pose}")

# Initialize the optimizer
optimizer = torch.optim.Adam([optim_pose], lr=0.001)
print(f"optimizer:\n {optimizer}")

### Define the loss function `loss_3d_keypoints`

In [ ]:
def loss_3d_keypoints(mano_joints, target_kpts):
    loss = mse_loss(mano_joints, target_kpts)
    loss /= mano_joints.shape[0]
    return loss


### Run the training loop

In [ ]:
loss_history = []
total_steps = 10000

for step in range(total_steps):
    optimizer.zero_grad()

    # Forward pass through the MANO layer
    mano_verts, mano_joints = mano_layer_forward(mano_layer, optim_pose, shape_params)

    # Compute the 3D keypoints loss
    loss = loss_3d_keypoints(mano_joints, target_kpts)

    # Backward pass (compute gradients)
    loss.backward()

    # Update MANO parameters (optimize)
    optimizer.step()

    # Print the loss
    if (step+1) % 500 == 0:
        print(f"Step {step+1:06d}/{total_steps:06d}, Loss {loss.item():11.8f}")

    # Save the loss
    loss_history.append(loss.item())

### Visualize the loss curve

In [ ]:
# Plot the loss history
plt.plot(loss_history)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Optimization Loss")
plt.show()


### Visualize the predicted 3D keypoints

In [ ]:
# Forward pass through the MANO layer to get the optimized hand mesh
mano_verts, mano_joints = mano_layer_forward(mano_layer, optim_pose, shape_params)
mano_faces = mano_layer.th_faces

# Convert the resulting vertices, joints and faces to numpy
mano_verts = mano_verts[0].detach().cpu().numpy()
mano_joints = mano_joints[0].detach().cpu().numpy()
mano_faces = mano_faces.detach().cpu().numpy()


### Visualize the MANO hand mesh

In [ ]:
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(mano_verts)
mesh.triangles = o3d.utility.Vector3iVector(mano_faces)
mesh.paint_uniform_color([0.3, 0.3, 0.3])

mesh.compute_vertex_normals()
mesh.normalize_normals()

# Joints as red spheres
joint_mesh = o3d.geometry.TriangleMesh()
for joint in mano_joints:
    joint_mesh += o3d.geometry.TriangleMesh.create_sphere(radius=0.003).translate(joint)
joint_mesh.paint_uniform_color([1.0, 0.0, 0.0])

# Keypoints as blue spheres
kpts_mesh = o3d.geometry.TriangleMesh()
for kpt in target_kpts[0].cpu().numpy():
    kpts_mesh += o3d.geometry.TriangleMesh.create_sphere(radius=0.003).translate(kpt)
kpts_mesh.paint_uniform_color([0.0, 0.0, 1.0])

o3d.visualization.draw([mesh, joint_mesh, kpts_mesh])

### Practice

- how about the MANO hand mesh visualization? when `lr=0.001, total_steps=1000`
- how about the MANO hand mesh visualization? when `lr=0.001, total_steps=3000`
- how about the MANO hand mesh visualization? when `lr=0.001, total_steps=5000`
- how about the MANO hand mesh visualization? when `lr=0.01, total_steps=1000`
- how about the MANO hand mesh visualization? when `lr=0.01, total_steps=3000`
- how about the MANO hand mesh visualization? when `lr=0.01, total_steps=5000`
- What will happen if we don't set the wrist joints to the target 3D keypoint? (Comment out the code ln:6 in `Initialize the optimizer "Adam"` section and run the training loop again)